<a href="https://colab.research.google.com/github/yaohodalo/SureStart2021/blob/main/Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Necessary packages
import pandas as pd
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import cross_val_score as cvs
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import keras
import tensorflow as tf
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError as mse
import sklearn.metrics
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import precision_score, recall_score

In [ ]:
#Getting pathnames for each file in the input folder
for dirname, _, filenames in os.walk('/content/Sarcasm_Headlines_Dataset_v2.json'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#Function to parse data
def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

#Taking in the data in one of the json files (I'm using the slightly larger one)
data = list(parse_data("/content/Sarcasm_Headlines_Dataset_v2.json"))

#Check 0-th index in our data
data[0]

{'article_link': 'https://www.theonion.com/thirtysomething-scientists-unveil-doomsday-clock-of-hai-1819586205',
 'headline': 'thirtysomething scientists unveil doomsday clock of hair loss',
 'is_sarcastic': 1}

In [ ]:
#Creating our X variable
vectorizer = TfidfVectorizer(max_features=50, use_idf=False)
headlines = [i['headline'] for i in data]
X = vectorizer.fit_transform(headlines).toarray()

#Creating our y variable
y = np.ravel([i['is_sarcastic'] for i in data])

#Creating a train and test split
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.2, random_state = 1693)

In [ ]:
#Now we're going to build the model with its layers

#Initialize the model
model = Sequential()

#Add the input layer
model.add(Dense(24, activation = 'relu', input_shape = (50,)))

#Add first hidden layer
model.add(Dense(12, activation = 'relu'))

#Add second hidden layer
model.add(Dense(8, activation = 'relu'))

#Add output layer
model.add(Dense(1, activation='softmax'))

In [ ]:
#Now we're going to compile the model
#Our loss function is binary crossentropy
#Our optimizer is adam

model.compile(loss = 'binary_crossentropy', 
              optimizer = 'adam',
              metrics = ['accuracy', 'mse'])

#We're going to also fit the model
#We're going to do 20 epochs
#The batch size will be 224 to get ~100 iterations per epoch
model.fit(X_train, y_train, epochs = 50,
          batch_size = 224, verbose = 1)

Epoch 1/50
103/103 [==============================] - 1s 1ms/step - loss: 0.6854 - accuracy: 0.4806 - mse: 0.5194
Epoch 2/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5778 - accuracy: 0.4750 - mse: 0.5250
Epoch 3/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5355 - accuracy: 0.4813 - mse: 0.5187
Epoch 4/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5319 - accuracy: 0.4789 - mse: 0.5211
Epoch 5/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5362 - accuracy: 0.4783 - mse: 0.5217
Epoch 6/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5242 - accuracy: 0.4732 - mse: 0.5268
Epoch 7/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5270 - accuracy: 0.4804 - mse: 0.5196
Epoch 8/50
103/103 [==============================] - 0s 1ms/step - loss: 0.5277 - accuracy: 0.4763 - mse: 0.5237
Epoch 9/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5249 - accur

In [ ]:
#Next, we'll test the model on the test dataset we set aside

#Prediction on the X_test data, round each to an integer (either 0 or 1)
y_pred = np.around(model.predict(X_test))

#We're going to now look at the accuracy and loss
score = model.evaluate(X_test, y_test, verbose=1)
print(score)

#We'll print precision and recall too
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")

179/179 [==============================] - 0s 999us/step - loss: 0.5406 - accuracy: 0.7121 - mse: 0.1829
[0.5406159162521362, 0.7120894193649292, 0.18290123343467712]
Precision: 0.6605052372150338
Recall: 0.7970260223048327


In [ ]:
#Now we're going to make a confusion matri
#The rows are the known labels, the columns are the predicted labels
matrix = cm(y_test, y_pred)
df = pd.DataFrame(columns = ['', 'is_sarcastic', 'not_sarcastic'])
df.loc[len(df)] = ['is_sarcastic', matrix[0][0], matrix[0][1]]
df.loc[len(df)] = ['not_sarcastic', matrix[1][0], matrix[1][1]]
print(df)

                 is_sarcastic not_sarcastic
0   is_sarcastic         1877          1157
1  not_sarcastic          515          2175
